# 1. Import Libraries

In [70]:
import torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from peft import PeftModel
import time

# 2. Load Model & Tokenizer

In [71]:
ADAPTER_PATH = "./results" 
BASE_MODEL_NAME = "google/flan-t5-base" 

device = "cuda" if torch.cuda.is_available() else "cpu"

base_model = AutoModelForSeq2SeqLM.from_pretrained(BASE_MODEL_NAME)
model = PeftModel.from_pretrained(base_model, ADAPTER_PATH).to(device)
tokenizer = AutoTokenizer.from_pretrained(ADAPTER_PATH)

# 3. Model Inference Functions

In [78]:
def generate_summary(text, style, model, tokenizer):
  model.eval()
  input_text = f"Summarize {style}: {text}"
  input_words = len(text.split())

  inputs = tokenizer(
    input_text,
    max_length=512,
    truncation=True,
    return_tensors='pt'
  ).to(device)

  if style == 'Harsh':
    max_len = int(input_words * 0.35)
    min_len = 5
    rep_penalty = 2.5
    length_penalty = 1.5
    beam_size = 4
    max_cap = 120
  elif style == 'Balanced':
    max_len = int(input_words * 0.50)
    min_len = 20
    rep_penalty = 1.5
    length_penalty = 1.2
    beam_size = 4
    max_cap = 180
  else:
    max_len = int(input_words * 0.70)
    min_len = 50
    rep_penalty = 1.2
    length_penalty = 0.8
    beam_size = 4
    max_cap = 256

  max_len = min(max_len, max_cap)

  with torch.no_grad():
    outputs = model.generate(
      input_ids=inputs['input_ids'],
      attention_mask=inputs['attention_mask'],
      max_length=max_len,
      min_length=min_len,
      num_beams=beam_size,
      length_penalty=length_penalty,
      repetition_penalty=rep_penalty,
      no_repeat_ngram_size=3,
      early_stopping=True
    )

  return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [79]:
styles = ["Harsh", "Balanced", "Detailed"]

def test_inference(text):
  print("Original Text Word Count:", len(text.split()))
  print("-" * 50)

  for style in styles:
    start_time = time.time()
    summary = generate_summary(text, style, model, tokenizer)
    end_time = time.time()
    
    print(f"\n=== {style} (Words: {len(summary.split())}) ===")
    print(f"Summary: {summary}")
    print(f"Time Taken: {end_time - start_time:.2f}s")

# 4. Model Inference

In [80]:
# Text Reference: https://www.bbc.com/news/articles/c9qe9wvq534o
text = '''
  Great white sharks in the Mediterranean Sea are in danger of disappearing, with illegal fishing contributing to their decline.
  This is according to research by US scientists, working in partnership with UK charity Blue Marine Foundation. They say some of the most threatened species - including great white sharks - are being sold in North African fish markets.
  Great whites are one of more than 20 Mediterranean shark species protected under international law, meaning it is illegal to fish for them or to sell them.
  By monitoring fishing ports on the Mediterranean coast of North Africa, however, researchers discovered that at least 40 great white sharks have been killed there in 2025 alone.
  The BBC has also found, and independently verified, footage from social media of protected sharks being brought dead into North African ports.
  One video showed a large great white being hauled ashore from a fishing boat in Algeria. Another, filmed in Tunisia, shows heads and fins of what appears to be a short-finned mako shark, which is also a threatened and protected species, being prepared for sale.
  Lead researcher, Dr Francesco Ferretti from the US university Virginia Tech, explained that many shark populations - white sharks in particular - had declined dramatically in the Mediterranean in recent decades.
  "No other stretch of water is fished like the Mediterranean Sea," he said, speaking to the BBC News science team while working on a research vessel off the coast of Sicily in late 2025.
  "The impact of industrial fishing has been intensifying... and it's plausible that they will go extinct in the near future."
  The Mediterranean white shark population is now classified as Critically Endangered by the International Union for the Conservation of Nature.
  In their latest attempt to find and study the predators, Dr Ferretti and his team worked in the Strait of Sicily - an area between Sicily and North Africa that has been identified as a "last stronghold" in the Mediterranean for several threatened shark species.
  One key aim of their mission was to fit a satellite tracking tag onto a white shark - something that has never been done in the Mediterranean Sea.
  To attempt this, the researchers brought more than three tonnes of fish bait - a shipping container packed with frozen mackerel and tuna scraps, as well as 500 litres of tuna oil to create a "fat slick" that many sharks would be able to smell from hundreds of metres away.
  Despite working for two weeks - baiting the ocean, taking samples of seawater to search for shark DNA and using underwater cameras - the researchers did not manage to find any animals to tag.
  They captured only a brief glimpse of one blue shark on their submarine cameras.
  "It's disheartening," Dr Ferretti told us. "It just shows how degraded this ecosystem is."
  While the team was searching for surviving sharks, they also received reports that a juvenile great white had been caught and killed in a North African fishery - just 20 nautical miles from where they were working.
'''

test_inference(text)

Original Text Word Count: 507
--------------------------------------------------

=== Harsh (Words: 17) ===
Summary: The number of great white sharks killed in North Africa has fallen dramatically, according to new research.
Time Taken: 1.27s

=== Balanced (Words: 40) ===
Summary: Great white sharks in the Mediterranean are in danger of disappearing, US scientists say. Some of the most threatened species are being sold in North African markets. Video shows great white being hauled ashore from a fishing boat in Algeria.
Time Taken: 2.91s

=== Detailed (Words: 96) ===
Summary: Great white sharks in the Mediterranean Sea are in danger of disappearing, according to research by US scientists. The BBC reports that at least 40 Great White sharks have been killed in North Africa in the last 2025 alone. "No other stretch of water is fished like the Mediterranean sea," says a US researcher. "The impact of industrial fishing has been intensifying ... and it's plausible that they will go extinct i

In [81]:
# Text Reference: https://en.wikipedia.org/wiki/Minecraft
text = '''
  Minecraft is a sandbox game developed and published by Mojang Studios. Formally released on 18 November 2011 for personal computers following its initial public alpha release on 17 May 2009, it has been ported to numerous platforms, including mobile devices and various video game consoles.
  In Minecraft, players explore a procedurally generated, three-dimensional world with virtually infinite terrain made up of voxels (cubes). Players can discover and extract raw materials, craft tools and items, and build structures, earthworks, and machines. Depending on the game mode, players can fight hostile mobs, as well as cooperate with or compete against other players in multiplayer. The game's large community offers a wide variety of user-generated content, such as modifications, servers, player skins, texture packs, and custom maps, which add new game mechanics and possibilities.
  Originally created in 2009 by Markus "Notch" Persson using the Java programming language, Jens "Jeb" Bergensten was handed control over the game's continuing development following its full release in 2011. In 2014, Mojang and the Minecraft intellectual property were purchased by Microsoft for US$2.5 billion; Xbox Game Studios hold the publishing rights for the Bedrock Edition, the cross-platform version based on the mobile Pocket Edition which replaced the existing console versions in 2017. Bedrock is updated concurrently with Mojang's original Java Edition, although with numerous, generally small, differences.
'''

test_inference(text)

Original Text Word Count: 219
--------------------------------------------------

=== Harsh (Words: 11) ===
Summary: Minecraft is an open-world sandbox video game developed by Mojang Studios.
Time Taken: 1.11s

=== Balanced (Words: 36) ===
Summary: Minecraft is a sandbox game developed and published by Mojang Studios. It has been ported to various platforms, including mobile devices and various video game consoles. The game's community offers a wide variety of user-generated content.
Time Taken: 2.88s

=== Detailed (Words: 97) ===
Summary: Minecraft is a sandbox game that's been ported to multiple platforms, including mobile devices, and Xbox Game Studios owns the publishing rights for the Bedrock Edition, a cross-platform version of the mobile Pocket Edition that replaced the existing console versions in 2017. The game was first released in 2011 for personal computers, but it's now available on mobile devices and other video game consoles, reports the Wall Street Journal. The game's m

In [82]:
# Text Reference: https://www.sciencedirect.com/science/article/pii/S2949719124000189
text = '''
  This paper explores the complex field of text summarization in Natural Language Processing (NLP), with particular attention to the development and importance of semantic understanding.
  Text summarization is a crucial component of natural language processing (NLP), which helps to translate large amounts of textual data into clear and understandable representations.
  As the story progresses, it demonstrates the dynamic transition from simple syntactic structures to sophisticated models with semantic comprehension.
  In order to effectively summarize, syntactic, semantic, and pragmatic concerns become crucial, highlighting the necessity of capturing not only grammar but also the context and underlying meaning.
  It examines the wide range of summarization models, from conventional extractive techniques to state-of-the-art tools like pre-trained models. Applications are found in many different fields, demonstrating how versatile summarizing techniques are.
  Semantic drift and domain-specific knowledge remain obstacles, despite progress. In the future, the study predicts developments like artificial intelligence integration and transfer learning, which motivates academics to investigate these prospects for advancement.
  The approach, which is based on the PRISMA framework, emphasizes a methodical and open literature review. The work attempts to further natural language processing (NLP) and text summarization by combining various research findings and suggesting future research directions in this dynamic subject.
'''

test_inference(text)

Original Text Word Count: 201
--------------------------------------------------

=== Harsh (Words: 12) ===
Summary: Text summarization in natural language processing: a methodical and open literature review
Time Taken: 1.11s

=== Balanced (Words: 34) ===
Summary: Text summarization is a crucial component of natural language processing. It demonstrates the dynamic transition from simple syntactic structures to sophisticated models with semantic comprehension. Semantic drift and domain-specific knowledge remain obstacles, despite progress.
Time Taken: 2.63s

=== Detailed (Words: 95) ===
Summary: Text summarization is a key component of natural language processing, and it's important to capture not only grammar but also the context and underlying meaning in order to effectively summarize large amounts of textual data. This paper explores the dynamic transition from simple syntactic structures to sophisticated models with semantic comprehension, the New York Times reports. It focuses on t

In [83]:
text = '''
  Apple announced a new version of its MacBook lineup on Tuesday, featuring faster processors and improved battery life. The company said the devices will be available worldwide starting next month.
'''

test_inference(text)

Original Text Word Count: 30
--------------------------------------------------

=== Harsh (Words: 7) ===
Summary: Apple has unveiled a new line of
Time Taken: 0.53s



=== Balanced (Words: 12) ===
Summary: Apple announced a new version of its MacBook lineup on Tuesday. The
Time Taken: 0.74s



=== Detailed (Words: 18) ===
Summary: Apple has announced a new version of its MacBook lineup, which will be available worldwide starting next month
Time Taken: 1.39s


In [84]:
text = '''
  The government announced a comprehensive policy package to accelerate renewable energy adoption nationwide. The plan includes tax incentives for solar and wind projects, funding for grid modernization, and support for energy storage research. Officials claim the initiative will reduce emissions and create jobs. Critics argue implementation details remain unclear, while supporters believe long-term environmental and economic benefits justify the investment.
'''

test_inference(text)

Original Text Word Count: 60
--------------------------------------------------

=== Harsh (Words: 14) ===
Summary: The government has announced plans to boost renewable energy adoption in England and Wales.
Time Taken: 1.04s

=== Balanced (Words: 24) ===
Summary: Government announces comprehensive policy package to accelerate renewable energy adoption. Plan includes tax incentives for solar and wind projects, funding for grid modernization. Cri
Time Taken: 1.68s



=== Detailed (Words: 35) ===
Summary: The US government has announced a comprehensive policy package to accelerate renewable energy adoption nationwide. The plan includes tax incentives for solar and wind projects, funding for grid modernization, and support for energy storage research,
Time Taken: 2.42s


In [85]:
text = '''
  The company reported steady revenue growth during the most recent quarter despite challenging market conditions. Executives attributed the performance to increased digital sales, improved supply chain management, and expansion into new regional markets. Rising operational costs and inflationary pressures partially offset gains, but leadership remains optimistic. Management plans to focus on efficiency improvements and targeted investments to sustain profitability. Analysts note that while growth has slowed compared to previous years, the firm’s diversified strategy positions it well to adapt to ongoing economic uncertainty and shifting consumer demand.
'''

test_inference(text)

Original Text Word Count: 87
--------------------------------------------------

=== Harsh (Words: 23) ===
Summary: Shares in BNP Paribas have fallen more than 4% after the firm reported a strong performance in the first half of the year
Time Taken: 1.73s

=== Balanced (Words: 31) ===
Summary: Revenue growth attributed to increased digital sales, improved supply chain management. Management plans to focus on efficiency improvements and targeted investments to sustain profitability. Growth has slowed compared to previous years.
Time Taken: 2.31s

=== Detailed (Words: 46) ===
Summary: Shares in Apple rose more than 4% in the first quarter of this year, according to the company's latest earnings report. The company reported steady revenue growth during the most recent quarter, despite challenging market conditions, reports the Wall Street Journal. The firm's digital sales surged
Time Taken: 3.36s
